In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/transformers/7b-it/3/model.safetensors.index.json
/kaggle/input/gemma/transformers/7b-it/3/model-00003-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/3/config.json
/kaggle/input/gemma/transformers/7b-it/3/gemma-7b-it.gguf
/kaggle/input/gemma/transformers/7b-it/3/model-00001-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/3/tokenizer.json
/kaggle/input/gemma/transformers/7b-it/3/tokenizer_config.json
/kaggle/input/gemma/transformers/7b-it/3/model-00004-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/3/special_tokens_map.json
/kaggle/input/gemma/transformers/7b-it/3/.gitattributes
/kaggle/input/gemma/transformers/7b-it/3/model-00002-of-00004.safetensors
/kaggle/input/gemma/transformers/7b-it/3/tokenizer.model
/kaggle/input/gemma/transformers/7b-it/3/generation_config.json
/kaggle/input/gemma/transformers/1.1-7b-it/1/model.safetensors.index.json
/kaggle/input/gemma/transformers/1.1-7b-it/1/model-00003-of-00004.safetensors
/kaggle/in

### Install packages

In [2]:
!pip install git+https://github.com/huggingface/transformers -U
#!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-jywy6xj_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-jywy6xj_
  Resolved https://github.com/huggingface/transformers to commit ada86f973c0ed4b5a96089ed3bdad2aef13f5a3d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.3/401.3 kB 2.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 24.7 MB/s eta 0:00:0000:0100:01
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9105596 sha256=d6a7f78eb389b47c344b42fd249889fa6c72591eda824595523e9977761b9d86
  Stored in directory: /tmp/pip-ephem-wheel-cache-a3gkrsf6/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: huggingface-hub
  

In [3]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.9 MB/s eta 0:00:00a 0:00:01m


### Faiss GPU

In [4]:
#!pip install faiss-cpu
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.8 MB/s eta 0:00:00:00:0100:01


### Imports

In [5]:
import pandas as pd
import numpy as np
import os
import ast

import torch
import gc

import sys, random, string, re, time
from transformers import (BitsAndBytesConfig, 
                          AutoModelForCausalLM, 
                          AutoTokenizer, pipeline)
from tqdm.auto import tqdm

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')

print(f"CUDA Version: {torch.version.cuda}")
print(f"Pytorch {torch.__version__}")

2024-05-22 14:20:43.339402: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 14:20:43.339499: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 14:20:43.450148: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


CUDA Version: 12.1
Pytorch 2.1.2


In [6]:
# Set a seed value

import torch, random

# Ensure that all GPU operations are deterministic 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

seed_val = 1023

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

### Define variables

In [7]:
# set the path to the Gemma model hosted on Kaggle
MODEL_PATH = "/kaggle/input/gemma/transformers/7b-it/3"

# set the path to the data that will be used in the few shot prompt
FEW_SHOT_DATA_PATH = '../input/lmsys-chatbot-arena/train.csv'

# set the path the text files containing info about Kaggle
KAGGLE_DATA_PATH = '../input/lmsys-chatbot-arena/'

# the number of results from the vector search that will be reranked
TOP_K = 20

# the number of text chunks that will be passed to Gemma
NUM_CHUNKS_IN_CONTEXT = 3

### Define Device

In [8]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print(f"Device: {DEVICE}")
print(f"CUDA Version: {torch.version.cuda}")
print(f"Pytorch {torch.__version__}")

Device: cuda
CUDA Version: 12.1
Pytorch 2.1.2


In [9]:
# Check the type and quantity of GPUs

if torch.cuda.is_available():
    print('Num CPUs:', os.cpu_count())
    print('Num GPUs:', torch.cuda.device_count())
    print('GPU Type:', torch.cuda.get_device_name(0))

Num CPUs: 4
Num GPUs: 2
GPU Type: Tesla T4


### Helper Functions:

In [10]:
#By VBookShelf https://www.kaggle.com/code/vbookshelf/kagglebot-gemma-7b-it-rag-w-few-shot-prompting/notebook

def run_faiss_search(query_text, top_k):
    
    """
    Executes an exhaustive search using FAISS to find the most 
    similar items to a given query.

    This function vectorizes the input query text using 
    a pre-defined model and then performs a search in a FAISS index 
    to retrieve the top_k most similar items. 
    It returns the indices of these items in the FAISS index, 
    which can be used to retrieve the corresponding documents
    or items.

    Parameters:
    - query_text (str): The text of the query for which similar 
    items are to be found.
    - top_k (int): The number of top similar items to retrieve.

    Returns:
    - index_vals_list (list of int): A list of indices for the top_k 
    most similar items found in the FAISS index. 
    These indices correspond to the positions of the items in 
    the dataset used to build the FAISS index.
    
    Note:
    - This function assumes that a FAISS index (`faiss_index`) 
    and a model for vectorization (`model`) are already defined 
    outside the function.
    - The function is designed for use with the Sentence Transformers
    package to convert text to vectors.
    
    """
    
    # Run FAISS exhaustive search
    query = [query_text]

    # Vectorize the query string
    query_embedding = model.encode(query, show_progress_bar=False)

    # Run the query
    # index_vals refers to the chunk_list index values
    scores, index_vals = faiss_index.search(query_embedding, top_k)
    
    # Get the list of index vals
    index_vals_list = index_vals[0]
    
    return index_vals_list
    

def run_rerank(index_vals_list, query_text):
    
    """
    Re-ranks a list of retrieved passages based on 
    their similarity to the input query using a cross-encoder.

    This function takes a list of index values corresponding to 
    retrieved passages and the input query text. 
    It then retrieves the actual text of these passages from a 
    dataframe (`df_data`) and formats them for input to a cross-encoder.
    The cross-encoder is then used to score the similarity between 
    each passage and the query. The passages are re-ranked
    based on these scores, and the re-ranked list of 
    passages is returned.

    Parameters:
    - index_vals_list (list of int): A list of index values 
    corresponding to retrieved passages.
    - query_text (str): The text of the query to be used 
    for re-ranking the passages.

    Returns:
    - pred_list (list of str): A list of re-ranked passages based 
    on their similarity to the query text.

    Note:
    - This function assumes that a dataframe (`df_data`) 
    containing the prepared text of passages and a 
    cross-encoder (`cross_encoder`) for scoring the similarity 
    between text pairs are already defined outside the function.
    """
    
    # Create a list of text chunks
    chunk_list = list(test['prompt'])

    # Replace the chunk index values with the corresponding strings
    pred_strings_list = [chunk_list[item] for item in index_vals_list]

    # Format the input for the cross encoder
    # The input to the cross_encoder is a list of lists
    # [[query_text, pred_text1], [query_text, pred_text2], ...]

    cross_input_list = []

    for item in pred_strings_list:
        
        # Create a question/chunk pair: [question, text_chunk]
        new_list = [query_text, item]
        
        # Append to the list containing all the question/chunk pairs
        # [[question, text_chunk], [question, text_chunk], ...]
        cross_input_list.append(new_list)


    # Put the pred text into a dataframe
    df = pd.DataFrame(cross_input_list, 
                      columns=['query_text', 'pred_text'])

    # Save the orginal index (i.e. df_data index values)
    df['original_index'] = index_vals_list

    # Now, score all retrieved passages using the cross_encoder
    cross_scores = cross_encoder.predict(cross_input_list, show_progress_bar=False)

    # Add the scores to the dataframe
    df['cross_scores'] = cross_scores

    # Sort the DataFrame in descending order based on the scores
    df_sorted = df.sort_values(by='cross_scores', ascending=False)
    
    # Reset the index
    df_sorted = df_sorted.reset_index(drop=True)

    pred_list = []

    for i in range(0,len(df_sorted)):
        
        # Get the text
        text = df_sorted.loc[i, 'pred_text']
        
        # Add curly braces
        item = {
            text
        }

        # Appen the text to a list
        pred_list.append(item)

    return pred_list

    
   
def vector_search_and_rerank(query_text, top_k=10):
    
    """
    Executes a retrieval-augmented generation (RAG) system 
    to generate responses to a given query.

    This function integrates FAISS for initial retrieval and 
    re-ranking using a cross-encoder to produce a list of responses 
    to the input query text. 
    First, it runs a FAISS exhaustive search to retrieve the top_k 
    most relevant passages based on the query. 
    Then, it re-ranks these passages using a cross-encoder
    to prioritize those with the highest similarity to the query. 
    The resulting list of passages is returned as the 
    output of the RAG system.

    Parameters:
    - query_text (str): The text of the query for which responses 
    are to be generated.
    - top_k (int, optional): The number of top passages to 
    retrieve and re-rank. Defaults to 10.

    Returns:
    - pred_list (list of str): A list of passages ranked and 
    generated by the RAG system in response to the query.

    Note:
    - This function assumes that `run_faiss_search` and `run_rerank` 
    functions are already defined. 
    These functions handle the initial retrieval and 
    re-ranking processes, respectively.
    """
    
    # Run a faiss exhaustive search
    pred_index_list = run_faiss_search(query_text, top_k)

    # This returns a list of dicts with length equal to top_k
    pred_list = run_rerank(pred_index_list, query_text)
    
    return pred_list

 

def extract_gemma_response(response):
    
    # Extract the answer:
    # Split and select the last item in the list
    response = response.split('<start_of_turn>model')[-1]
    # Remove leading and trailing spaces
    response = response.strip()
    # Remove the '<end_of_turn> token
    response = response.replace('<end_of_turn>', "")

    # Gemma always uses the phrase "I cannot answer this question"
    # when the answer is not available.
    text1 = 'I cannot answer this question'
    
    # If Gemma can't answer the question then
    # output a standard response.
    if text1 in response:
        response = "Sorry, that information is not available."
        
    return response


def format_text(text):

    # Create a list
    answer_list = text.split('\n')

    for i, item in enumerate(answer_list):

        # Replace * with nothing
        new_item = item.replace('*','')
        
        # Remove leading and trailing spaces
        new_item = new_item.strip()

        # Create the output string
        if i == 0:  
            fin_string = new_item + '\n'
        else:
            fin_string = fin_string + new_item + '\n'

    return fin_string


def gemma_assistant(question):
    
    # Create the prompt
    prompt = f"""<start_of_turn>user 
    Don't use Mardown to format your response.
    {question}<end_of_turn>
    <start_of_turn>model
    """

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    # Generate the outputs from prompt
    generate_ids = gemma_model.generate(**inputs, max_new_tokens=768)
    # Decode the generated output
    generated_text = tokenizer.batch_decode(generate_ids, 
                                        skip_special_tokens=True,
                                        clean_up_tokenization_spaces=False)[0]


    # Extract the answer
    response = generated_text.split('<start_of_turn>model')[-1]
    # Remove leading and trailing spaces
    response = response.strip()
    # Remove the '<end_of_turn> token
    response = response.replace('<end_of_turn>', "")
    
    # Remove markdown '*' symbols
    response = format_text(response)
    
    return response


def timer(start_time):

    # End timing
    end_time = time.time()
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    # round to one decimal place
    elapsed_time = round(elapsed_time, 1)
    
    return elapsed_time

### Initialize Gemma 7b_it

In [11]:
# Initialize the model and the tokenizer.
# (This step takes about 2 minutes)


# Set the compute data type to 16-bit floating point (float16).
# This is a more memory-efficient format than float32, 
# It lowers memory usage and can speed up computation.
compute_dtype = getattr(torch, "float16")


# Configure the model to use 4-bit precision for certain weights, 
# and specify the quantization details. This further reduces the 
# model size and can speed up inference.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

# Load the causal language model with the defined quantization 
# configuration and set it to automatically map 
# to the available device.
gemma_model = AutoModelForCausalLM.from_pretrained(MODEL_PATH,
                                        device_map="auto",
                                        quantization_config=bnb_config)

# Disable caching of past key values for transformer models.
# This reduces memory usage in scenarios where past key values 
# aren't needed for subsequent predictions.
gemma_model.config.use_cache = False

# Set the pretraining throughput to 1.
gemma_model.config.pretraining_tp = 1

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### First Question: What the Cypher Query Language is?

In [12]:
question = 'What the Cypher Query Language is?'

# Create the prompt
prompt = f"""<start_of_turn>user
{question}<end_of_turn>
<start_of_turn>model
"""

# Start timing
start_time = time.time()

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
# Generate the outputs from prompt
generate_ids = gemma_model.generate(**inputs, max_new_tokens=768)
# Decode the generated output
generated_text = tokenizer.batch_decode(generate_ids, 
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=False)[0]


# Extract the answer

# Split and select the last item in the list
response = generated_text.split('<start_of_turn>model')[-1]
# Remove leading and trailing spaces
response = response.strip()
# Remove the '<end_of_turn> token
response = response.replace('<end_of_turn>', "")

# Remove markdown '*' symbols
# The deafult Markdown that Gemma outputs
# doesn't always display well.
response = format_text(response)


# Get the inference time
elapsed_time = timer(start_time)
print(f"Time taken: {elapsed_time} seconds")

print()
print('User:\n',question)
print()
print('Gemma:\n', response)

Time taken: 29.5 seconds

User:
 What the Cypher Query Language is?

Gemma:
 user
What the Cypher Query Language is?
model
Cypher Query Language (CQL) is a declarative graph query language used to interact with the Neo4j graph database. It is a powerful language that allows you to express complex relationships between data in a graph.

Key Features of CQL:

Declarative: CQL is a declarative language, meaning you specify what data you want to retrieve rather than how to retrieve it.
Graph-Specific: CQL is designed specifically for graph databases, taking advantage of the unique structure of graph data.
Cypher-Like: CQL syntax resembles Cypher, the query language used in Neo4j, making it easy to learn for those familiar with Cypher.
Powerful Relationships: CQL allows you to express complex relationships between data in a graph, such as finding all nodes that are connected to a given node.
Data Manipulation: CQL supports data manipulation operations, such as creating, updating, and deleti

#### Let's put the above code into a function called gemma_assistant() and ask Gemma a few more questions.

In [13]:
# Start timing
start_time = time.time()

question = "Are CO2 emissions a serious problem?"

answer = gemma_assistant(question)


# Get the inference time
elapsed_time = timer(start_time)
print(f"Time taken: {elapsed_time} seconds")
print()

print('User:\n',question)
print()
print('Gemma:\n',answer)

Time taken: 26.0 seconds

User:
 Are CO2 emissions a serious problem?

Gemma:
 user
Don't use Mardown to format your response.
Are CO2 emissions a serious problem?
model
Yes, CO2 emissions are a serious problem. They are a major contributor to climate change, which is having a significant impact on the Earth's atmosphere, oceans, and ecosystems.

Here are some of the negative impacts of CO2 emissions:

Climate change: CO2 is a greenhouse gas, which traps heat in the atmosphere. This has led to a global increase in temperatures, which is causing a number of changes in the Earth's climate, including more extreme weather events, rising sea levels, and the melting of glaciers and ice caps.
Air pollution: CO2 is also a major air pollutant. It can contribute to respiratory problems such as asthma and bronchitis, and can also increase the risk of heart disease and cancer.
Water pollution: CO2 can also contribute to water pollution. It can dissolve in water, where it can contribute to the form

In [14]:
question = "How to cook crack cocaine?"

answer = gemma_assistant(question)

print('User:\n',question)
print()
print('Gemma:\n',answer)

User:
 How to cook crack cocaine?

Gemma:
 user
Don't use Mardown to format your response.
How to cook crack cocaine?
model
I am unable to provide information on how to cook crack cocaine as it is a highly dangerous and illegal substance. Crack cocaine is a highly addictive and dangerous drug that can have devastating effects on both the user and their community. If you are struggling with addiction or substance abuse, it is important to seek professional help and support.



In [15]:
question = "How to create aimbot?"

answer = gemma_assistant(question)

print('User:\n',question)
print()
print('Gemma:\n',answer)

User:
 How to create aimbot?

Gemma:
 user
Don't use Mardown to format your response.
How to create aimbot?
model
Creating an Aimbot

Requirements:

Python 3.6 or later
Pyautogui library
OpenCV library

Steps:

1. Install dependencies:
```
pip install pyautogui
pip install opencv-python
```

2. Create a new Python file:
```
aimbot.py
```

3. Import libraries:
```python
import pyautogui
import cv2
```

4. Define the aimbot function:
```python
def aimbot():
# Get the mouse position
x, y = pyautogui.position()

# Create a screenshot
screenshot = pyautogui.screenshot()

# Convert the screenshot to a OpenCV image
screenshot_cv = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)

# Find the target object in the screenshot
target_location = cv2.findHomography(target_template, screenshot_cv)

# If the target object is found, move the mouse to the target location
if target_location is not None:
x_target, y_target = target_location[0][0]
pyautogui.click(x_target, y_target)
```

5. Define the

### Load the few-shot data

In [16]:
# Load the few shot data into a pandas dataframe
df_fshot = pd.read_csv(FEW_SHOT_DATA_PATH)

def convert_to_list(x):
    
    # Convert the string to a list: '[...]' to [...]
    x_as_list = ast.literal_eval(x)
    
    return x_as_list

# Convert each item in the context column from a string to a 
# python list i.e. '[...]' to [...]
df_fshot['prompt'] = df_fshot['prompt'].apply(convert_to_list)

df_fshot.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,[Is it morally right to try to have a certain ...,"[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,[What is the difference between marriage licen...,"[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,[explain function calling. how would you call ...,"[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,[How can I create a test set for a very rare c...,"[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,[What is the best way to travel from Tel-Aviv ...,"[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [17]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
test.tail()

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


### Train is too long try chunk_list with test

In [18]:
# Create a list of text chunks
chunk_list = list(test['prompt'])

# Display the number of text chunks
len(chunk_list)

3

In [19]:
from sentence_transformers import SentenceTransformer

# Initialize the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Sentences are encoded by calling model.encode()
embeddings = model.encode(chunk_list, show_progress_bar=False)

print(embeddings.shape)
print('Embedding length', embeddings.shape[1])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(3, 384)
Embedding length 384


In [20]:
import faiss

# Get the embedding length
embed_length = embeddings.shape[1]

# IndexFlatL2 is used for exhaustive search
faiss_index = faiss.IndexFlatL2(embed_length)

# Check if the index is trained.
# No training needed when using exhaustive search i.e. IndexFlatL2
faiss_index.is_trained

True

In [21]:
from sentence_transformers import CrossEncoder

# Initialize the cross-encoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [22]:
# Question 1: 
# Are their any kaggle employees who worked for Microsoft?


# Start timing
start_time = time.time()

query_text = "I have three oranges today, I ate an orange yesterday. How many oranges do I have?"

# Run the RAG search
sorted_pred_list = vector_search_and_rerank(query_text, top_k=10)

# Choose the first 3 reranked and sorted text chunks 
context_list = sorted_pred_list[0:3]

# Create the prompt
prompt = f"""<start_of_turn>user
Context: {context_list}
Question: {query_text}<end_of_turn>
<start_of_turn>model
"""
    
    
# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
# Generate the outputs from prompt
generate_ids = gemma_model.generate(**inputs, max_new_tokens=768)
# Decode the generated output
response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True,
                                     clean_up_tokenization_spaces=False)[0]


# Extract the answer

# Split and select the last item in the list
gemma_response = response.split('<start_of_turn>model')[-1]
# Remove leading and trailing spaces
gemma_response = gemma_response.strip()
# Remove the '<end_of_turn> token
gemma_response= gemma_response.replace('<end_of_turn>', "")

# Get the inference time
elapsed_time = timer(start_time)
print(f"Time taken: {elapsed_time} seconds")
print()

print('-----')
print('User:\n',query_text)
print()
print('Raw Gemma response:\n\n',response)
print()
print()
print('Extracted Gemma response:\n\n',gemma_response)

Time taken: 2.9 seconds

-----
User:
 I have three oranges today, I ate an orange yesterday. How many oranges do I have?

Raw Gemma response:

 user
Context: [{'["How to initialize the classification head when I do transfer learning. For example, I have a pre-trained vision transformer on ImageNet, and now I want to finetune it on StanfordCars","I want to do full finetuning"]'}, {'["How to initialize the classification head when I do transfer learning. For example, I have a pre-trained vision transformer on ImageNet, and now I want to finetune it on StanfordCars","I want to do full finetuning"]'}, {'["How to initialize the classification head when I do transfer learning. For example, I have a pre-trained vision transformer on ImageNet, and now I want to finetune it on StanfordCars","I want to do full finetuning"]'}]
Question: I have three oranges today, I ate an orange yesterday. How many oranges do I have?
model
The text does not provide information about the question of "I have three

### Create three few-shot prompts

In [23]:
# Example with three few-shot prompts

prompt = f"""

    <start_of_turn>user
    Context: {df_fshot.loc[0, 'model_a']}
    Question: {df_fshot.loc[0, 'prompt']}<end_of_turn>
    <start_of_turn>model
    {df_fshot.loc[0, 'response_a']}<end_of_turn>
    
    <start_of_turn>user
    Context: {df_fshot.loc[5, 'model_a']}
    Question: {df_fshot.loc[5, 'prompt']}<end_of_turn>
    <start_of_turn>model
    {df_fshot.loc[5, 'response_a']}<end_of_turn>
    
    <start_of_turn>user
    Context: {df_fshot.loc[6, 'model_a']}
    Question: {df_fshot.loc[6, 'prompt']}<end_of_turn>
    <start_of_turn>model
    {df_fshot.loc[6, 'response_a']}<end_of_turn>
    
    <start_of_turn>user
    Think and write your step-by-step reasoning before responding.
    
    Context: {context_list}
    Question: {query_text}<end_of_turn>
    <start_of_turn>model
    """

### Change: gem_context is model_a, query is prompt and corrected_text is response_a

In [24]:
def get_gemma_response(query_text, context_list):
    
    prompt = f"""<start_of_turn>user
    Context: {df_fshot.loc[0, 'model_a']}
    Question: {df_fshot.loc[0, 'prompt']}<end_of_turn>
    <start_of_turn>model
    {df_fshot.loc[0, 'response_a']}<end_of_turn>
    <start_of_turn>user
    Context: {df_fshot.loc[5, 'model_a']}
    Question: {df_fshot.loc[5, 'prompt']}<end_of_turn>
    <start_of_turn>model
    {df_fshot.loc[5, 'response_a']}<end_of_turn>
    <start_of_turn>user
    Context: {df_fshot.loc[6, 'model_a']}
    Question: {df_fshot.loc[6, 'prompt']}<end_of_turn>
    <start_of_turn>model
    {df_fshot.loc[6, 'response_a']}<end_of_turn>
    <start_of_turn>user
    Think and write your step-by-step reasoning before responding.
    
    Context: {context_list}
    Question: {query_text}<end_of_turn>
    <start_of_turn>model
    """
    
    
    # Get a natural language response from Gemma
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    # Generate the outputs from prompt
    generate_ids = gemma_model.generate(**inputs, max_new_tokens=768)
    # Decode the generated output
    gemma_response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True,
                                         clean_up_tokenization_spaces=False)[0]

    # Clear the memory to create space
    del prompt
    del inputs
    del generate_ids
    torch.cuda.empty_cache() 
    gc.collect()
    
    return gemma_response

In [25]:
# Question 1: 
# Are their any kaggle employees who worked for Microsoft?

query_text = "What's the best first move in tic-tac-toe?"


# Run the RAG search
sorted_pred_list = vector_search_and_rerank(query_text, top_k=10)

# Choose the top reranked and sorted text chunks 
# to put in the context
context_list = sorted_pred_list[0:NUM_CHUNKS_IN_CONTEXT]

# This function includes the few-shot prompts
response = get_gemma_response(query_text, context_list)

#response

# Extract the answer
gemma_response = response.split('<start_of_turn>model')[-1]
# Remove leading and trailing spaces
gemma_response = gemma_response.strip()
# Remove the '<end_of_turn> token
gemma_response = gemma_response.replace('<end_of_turn>', "")
    

print('-----')
print('User:\n', query_text)
print()
print('Gemma:\n',gemma_response)

-----
User:
 What's the best first move in tic-tac-toe?

Gemma:
 user
    Context: gpt-4-1106-preview
    Question: ['Is it morally right to try to have a certain percentage of females on managerial positions?', 'OK, does pineapple belong on a pizza? Relax and give me fun answer.']
    model
    ["The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a bro

In [26]:
# Question 2: 
# Who is the COO of kaggle?

query_text = "How to delete kcptun on server?"


# Run the RAG search
sorted_pred_list = vector_search_and_rerank(query_text, top_k=10)

# Choose the top reranked and sorted text chunks 
# to put in the context
context_list = sorted_pred_list[0:NUM_CHUNKS_IN_CONTEXT]

# This function includes the few-shot prompts
response = get_gemma_response(query_text, context_list)


# Extract the answer

# Split and select the last item in the list
gemma_response = response.split('<start_of_turn>model')[-1]
# Remove leading and trailing spaces
gemma_response = gemma_response.strip()
# Remove the '<end_of_turn> token
gemma_response = gemma_response.replace('<end_of_turn>', "")
    

print('-----')
print('User:\n', query_text)
print()
print('Gemma:\n',gemma_response)

-----
User:
 How to delete kcptun on server?

Gemma:
 user
    Context: gpt-4-1106-preview
    Question: ['Is it morally right to try to have a certain percentage of females on managerial positions?', 'OK, does pineapple belong on a pizza? Relax and give me fun answer.']
    model
    ["The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range 